In [24]:
import pandas as pd
import numpy as np
from gekko import GEKKO

In [25]:
# Load the CSV files into DataFrames
df_intake = pd.read_csv("Nutrient_Intake_Recommendations_Formalized.csv")
df_recipe_cost = pd.read_csv("Recipe_Cost_Formalized.csv")
df_recipe_nutrition = pd.read_csv("Recipe_Nutritional_Value_Formalized.csv")

sample example

In [36]:
from gekko import GEKKO

# Initialize Model
m = GEKKO(remote=False)
m.options.SOLVER=1
m.solver_options = [#maximum number of nlp solutions from the branch and bound method.
                    #A successful solution is returned if there is an integer solution upon reaching the maximum number of iterations.
                    #Otherwise, the solution is not considered to be successful and an error message is returned with the failed solution.
                    'minlp_maximum_iterations 10000', \
                    # minlp iterations with integer solution
                    'minlp_max_iter_with_int_sol 10', \
                    # maximum number of iterations for each nlp sub-problem.
                    'nlp_maximum_iterations 500', \
                    # 1 means solve minlp problem as a continuous nlp problem, ignoring integer constraints
                    'minlp_as_nlp 0', \
                     #- 1=depth first (find integer solution faster), 2=breadth first, 3=lowest objective leaf, 4=highest objective leaf
                    #* maximum deviation from whole number
                    'minlp_branch_method 3', \
                    'minlp_integer_tol 0.1', \
                    #amount that a candidate solution variable can deviate from an integer solution and still be considered an integer.
                    # covergence tolerance:gap is 
                    #the spread between the lowest candidate leaf (obj_r=non-integer solution) 
                    #and the best integer solution (obj_i). When the gap is below the minlp_gap_tol, 
                    #the best integer solution is returned.
                    'minlp_gap_tol 0.5'
                   ]
#help(m)

#define parameter
eq = m.Param(value=40)

# m.solver_options = ['minlp_maximum_iterations 10000*2']
# m.solver_options = ['minlp_integer_tol 1',\
#                     'minlp_gap_tol 1.0e-2']
#initialize variables
# m.Var(lb=0, integer=True)
x1,x2,x3,x4 = [m.Var(integer=True) for i in range(4)]

#initial values
x1.value = 1
x2.value = 5
x3.value = 5
x4.value = 1

#lower bounds
x1.lower = 1
x2.lower = 1
x3.lower = 1
x4.lower = 1

#upper bounds
x1.upper = 5
x2.upper = 5
x3.upper = 5
x4.upper = 5

#Equations
m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==eq)

#Objective
m.Obj(x1*x4*(x1+x2+x3)+x3)

#Set global options
m.options.IMODE = 3 #steady state optimization

#Solve simulation
m.solve(disp=False) # solve on public server

#Results
print('')
print('Results')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
print('x4: ' + str(x4.value))


Results
x1: [2.0]
x2: [4.0]
x3: [2.0]
x4: [4.0]


Prediction for one meal

In [40]:
import pandas as pd
from gekko import GEKKO

# Load the CSV files into DataFrames
df_intake = pd.read_csv("Nutrient_Intake_Recommendations_Formalized.csv")
df_recipe_cost = pd.read_csv("Recipe_Cost_Formalized.csv")
df_recipe_nutrition = pd.read_csv("Recipe_Nutritional_Value_Formalized.csv")

# Extract required data
time_available = 60*1.3 # Example time available (in minutes)

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Initialize gekko model
m = GEKKO(remote=False)
m.options.SOLVER=1
m.solver_options = [#maximum number of nlp solutions from the branch and bound method.
                    #A successful solution is returned if there is an integer solution upon reaching the maximum number of iterations.
                    #Otherwise, the solution is not considered to be successful and an error message is returned with the failed solution.
                    'minlp_maximum_iterations 10000', \
                    # maximum number of nlp solutions when a candidate integer solution is found
                    'minlp_max_iter_with_int_sol 500', \
                    # maximum number of iterations for each nlp sub-problem. lowever value usually save more time
                    'nlp_maximum_iterations 500', \
                    # 1 means solve minlp problem as a continuous nlp problem, ignoring integer constraints
                    'minlp_as_nlp 0', \
                    #- 1=depth first (find integer solution faster), 2=breadth first, 3=lowest objective leaf, 4=highest objective leaf
                    #* maximum deviation from whole number
                    'minlp_branch_method 3', \
                    #amount that a candidate solution variable can deviate from an integer solution and still be considered an integer.
                    'minlp_integer_tol 0.01', \
                    # covergence tolerance:gap is 
                    #the spread between the lowest candidate leaf (obj_r=non-integer solution) 
                    #and the best integer solution (obj_i). When the gap is below the minlp_gap_tol, 
                    #the best integer solution is returned.
                    'minlp_gap_tol 0.01',\
                    # convergence tolerance for the objective function. 
                    'objective_convergence_tolerance 1.0e-6'
                   ]

# Define decision variables
recipes_to_produce = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]

# Define the optimization problem
m.Maximize(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * recipes_to_produce[i] for i, recipe in enumerate(df_recipe_cost['Name'])))

# Constraint 1: time cost
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_available)

# Constraint 2: nutritional requirements
for nutrient in nutrient_requirements:
    m.Equation(sum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] * recipes_to_produce[i] for i, recipe in enumerate(df_recipe_nutrition['Name'])) >= nutrient_requirements[nutrient]/3)

# Solve the optimization problem
m.solve(disp=False)

# Print the results
print("Objective:", -round(m.options.objfcnval, 2))

# Get the quantities and round them to the nearest integer
quantities = [recipe.value[0] for recipe in recipes_to_produce]

# Output dictionary
output = dict(zip(list(df_recipe_cost['Name']), quantities))

# Filter out dishes with a number to make of 0.0
output = {dish: quantity for dish, quantity in output.items() if quantity > 0}

# Calculate time spent for each meal
time_spent = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities[i] for i, recipe in enumerate(df_recipe_cost['Name']))
# Print out time spent for each meal
print("Time spent:", round(time_spent, 2), "minutes")
# Print the filtered output
output

Objective: 9.0
Time spent: 75.0 minutes


{'Broccolini Quinoa Pilaf': 1.0,
 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}

Experiments with the exponential decay 

In [71]:
import pandas as pd
#define decay factor and decay factor multiplier
decay=0.4
decay_mul=1-decay

# Extract required data
time_available = 60*4  # Example time available (in minutes)

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Initialize gekko model
m = GEKKO(remote=False)
m.options.SOLVER=1

# Define decision variables
recipes_to_produce = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]

# Define the optimization problem  ((1-decay_mul**recipes_to_produce[recipe])/(1-decay_mul))
m.Maximize(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * (1-decay_mul**recipes_to_produce[i]) / (1-decay_mul) for i, recipe in enumerate(df_recipe_cost['Name'])))

# Constraint 1: time cost
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_available)

# Constraint 2: nutritional requirements
for nutrient in nutrient_requirements:
    m.Equation(sum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] * recipes_to_produce[i] for i, recipe in enumerate(df_recipe_nutrition['Name'])) >= nutrient_requirements[nutrient]/3)

# Solve the optimization problem
m.solve(disp=False)

# Print the results
print("Objective:", -round(m.options.objfcnval, 2))

# Get the quantities and round them to the nearest integer
quantities = [recipe.value[0] for recipe in recipes_to_produce]

# Output dictionary
output = dict(zip(list(df_recipe_cost['Name']), quantities))

# Filter out dishes with a number to make of 0.0
output = {dish: quantity for dish, quantity in output.items() if quantity > 0}
# Calculate time spent for each meal
time_spent = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities[i] for i, recipe in enumerate(df_recipe_cost['Name']))
# Print out time spent for each meal
print("Time spent:", round(time_spent, 2), "minutes")

output

Objective: 27.0
Time spent: 240.0 minutes


{'Beef Braised In Red Wine': 1.0,
 'Broccolini Quinoa Pilaf': 2.0,
 'Classic Carrot Cake With Cream Cheese Frosting': 1.0,
 'Easy Eclairs': 1.0,
 'Knishes - Potato Filling': 1.0}

Prediction for three meals. Namely breakfast, lunch and dinner

In [42]:
# Define the time available for each meal
time_breakfast = 60*0.5  # Example time available for breakfast (in minutes)
time_lunch = 60*1  # Example time available for lunch (in minutes)
time_dinner = 60*2  # Example time available for dinner (in minutes)

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Initialize gekko model
m = GEKKO(remote=False)
m.options.SOLVER=1
m.solver_options = [#maximum number of nlp solutions from the branch and bound method.
                    #A successful solution is returned if there is an integer solution upon reaching the maximum number of iterations.
                    #Otherwise, the solution is not considered to be successful and an error message is returned with the failed solution.
                    'minlp_maximum_iterations 10000', \
                    # maximum number of nlp solutions when a candidate integer solution is found
                    'minlp_max_iter_with_int_sol 500', \
                    # maximum number of iterations for each nlp sub-problem. lowever value usually save more time
                    'nlp_maximum_iterations 500', \
                    # 1 means solve minlp problem as a continuous nlp problem, ignoring integer constraints
                    'minlp_as_nlp 0', \
                    #- 1=depth first (find integer solution faster), 2=breadth first, 3=lowest objective leaf, 4=highest objective leaf
                    #* maximum deviation from whole number
                    'minlp_branch_method 3', \
                    #amount that a candidate solution variable can deviate from an integer solution and still be considered an integer.
                    'minlp_integer_tol 0.01', \
                    # covergence tolerance:gap is 
                    #the spread between the lowest candidate leaf (obj_r=non-integer solution) 
                    #and the best integer solution (obj_i). When the gap is below the minlp_gap_tol, 
                    #the best integer solution is returned.
                    'minlp_gap_tol 0.01',\
                    # convergence tolerance for the objective function. 
                    'objective_convergence_tolerance 1.0e-6'
                   ]
# Define decision variables for each meal
recipes_to_produce_breakfast = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]
recipes_to_produce_lunch = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]
recipes_to_produce_dinner = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]

# Define the optimization problem
m.Maximize(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * 
                (recipes_to_produce_breakfast[i] + recipes_to_produce_lunch[i] + recipes_to_produce_dinner[i])
                for i, recipe in enumerate(df_recipe_cost['Name'])))

# Constraint 1: time cost for each meal
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_breakfast[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_breakfast)
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_lunch[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_lunch)
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_dinner[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_dinner)

# Constraint 2: nutritional requirements: we want that combination all three meals together satisfy the nutritional requirements
for nutrient in nutrient_requirements:
    m.Equation(sum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] *
                    (recipes_to_produce_breakfast[i] + recipes_to_produce_lunch[i] + recipes_to_produce_dinner[i])
                    for i, recipe in enumerate(df_recipe_nutrition['Name'])) >= nutrient_requirements[nutrient])

# Solve the optimization problem
m.solve(disp=False)

# Print the results
print("Objective:", -round(m.options.objfcnval, 2))

# Get the quantities and round them to the nearest integer
quantities_breakfast = [recipe.value[0] for recipe in recipes_to_produce_breakfast]
quantities_lunch = [recipe.value[0] for recipe in recipes_to_produce_lunch]
quantities_dinner = [recipe.value[0] for recipe in recipes_to_produce_dinner]

# Output dictionaries
output_breakfast = dict(zip(list(df_recipe_cost['Name']), quantities_breakfast))
output_lunch = dict(zip(list(df_recipe_cost['Name']), quantities_lunch))
output_dinner = dict(zip(list(df_recipe_cost['Name']), quantities_dinner))


# Filter out dishes with a number to make of 0.0
output_breakfast = {dish: quantity for dish, quantity in output_breakfast.items() if quantity > 0}
output_lunch = {dish: quantity for dish, quantity in output_lunch.items() if quantity > 0}
output_dinner = {dish: quantity for dish, quantity in output_dinner.items() if quantity > 0}
# Calculate time spent for each meal
time_spent_breakfast = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_breakfast[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_lunch = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_lunch[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_dinner = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_dinner[i] for i, recipe in enumerate(df_recipe_cost['Name']))

# Print out time spent for each meal
print("Time spent for breakfast:", round(time_spent_breakfast, 2), "minutes")
print("Time spent for lunch:", round(time_spent_lunch, 2), "minutes")
print("Time spent for dinner:", round(time_spent_dinner, 2), "minutes")


Objective: 37.0
Time spent for breakfast: 30.0 minutes
Time spent for lunch: 60.0 minutes
Time spent for dinner: 111.0 minutes


In [43]:
print("breakfast")
print(output_breakfast)
print("lunch")
print(output_lunch)
print("dinner")
print(output_dinner)

breakfast
{'Broccolini Quinoa Pilaf': 1.0}
lunch
{'Broccolini Quinoa Pilaf': 2.0}
dinner
{'Slow Cooker Chicken and Dumplings': 6.0, 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}


Prediction for three meals with exponential decay

In [47]:
decay=0.4
decay_mul=1-decay

# Define the time available for each meal
time_breakfast = 60*0.5  # Example time available for breakfast (in minutes)
time_lunch = 60*1  # Example time available for lunch (in minutes)
time_dinner = 60*2  # Example time available for dinner (in minutes)

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Initialize gekko model
m = GEKKO(remote=False)
m.options.SOLVER=1
# we have to extend the max_itr since it cannot satisfied easily the convergence tol with default max iteration which is 10000
m.solver_options = [#maximum number of nlp solutions from the branch and bound method.
                    #A successful solution is returned if there is an integer solution upon reaching the maximum number of iterations.
                    #Otherwise, the solution is not considered to be successful and an error message is returned with the failed solution.
                    'minlp_maximum_iterations 10000', \
                    # maximum number of nlp solutions when a candidate integer solution is found
                    'minlp_max_iter_with_int_sol 5000', \
                    # maximum number of iterations for each nlp sub-problem. lowever value usually save more time
                    'nlp_maximum_iterations 200', \
                    # 1 means solve minlp problem as a continuous nlp problem, ignoring integer constraints
                    'minlp_as_nlp 0', \
                    #- 1=depth first (find integer solution faster), 2=breadth first, 3=lowest objective leaf, 4=highest objective leaf
                    #* maximum deviation from whole number
                    'minlp_branch_method 3', \
                    #amount that a candidate solution variable can deviate from an integer solution and still be considered an integer.
                    'minlp_integer_tol 0.06', \
                    # covergence tolerance:gap is 
                    #the spread between the lowest candidate leaf (obj_r=non-integer solution) 
                    #and the best integer solution (obj_i). When the gap is below the minlp_gap_tol, 
                    #the best integer solution is returned.
                    'minlp_gap_tol 0.1',\
                    # convergence tolerance for the objective function. 
                    'objective_convergence_tolerance 1.0e-2'
                   ]

# multiple options as one list

# Define decision variables for each meal
recipes_to_produce_breakfast = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]
recipes_to_produce_lunch = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]
recipes_to_produce_dinner = [m.Var(lb=0, integer=True) for _ in df_recipe_cost['Name']]

# Define the optimization problem
m.Maximize(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * 
               (1-decay_mul**(recipes_to_produce_breakfast[i] + recipes_to_produce_lunch[i] + recipes_to_produce_dinner[i])) / (1-decay_mul) 
                for i, recipe in enumerate(df_recipe_cost['Name'])))


# Constraint 1: time cost for each meal
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_breakfast[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_breakfast)
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_lunch[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_lunch)
m.Equation(sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_dinner[i] for i, recipe in enumerate(df_recipe_cost['Name'])) <= time_dinner)

# Constraint 2: nutritional requirements: we want that combination all three meals together satisfy the nutritional requirements
for nutrient in nutrient_requirements:
    m.Equation(sum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] *
                    (recipes_to_produce_breakfast[i] + recipes_to_produce_lunch[i] + recipes_to_produce_dinner[i])
                    for i, recipe in enumerate(df_recipe_nutrition['Name'])) >= nutrient_requirements[nutrient])

# Solve the optimization problem
m.solve()

# Print the results
print("Maximized preference:", -round(m.options.objfcnval, 2))

# Get the quantities and round them to the nearest integer
quantities_breakfast = [recipe.value[0] for recipe in recipes_to_produce_breakfast]
quantities_lunch = [recipe.value[0] for recipe in recipes_to_produce_lunch]
quantities_dinner = [recipe.value[0] for recipe in recipes_to_produce_dinner]

# Output dictionaries
output_breakfast = dict(zip(list(df_recipe_cost['Name']), quantities_breakfast))
output_lunch = dict(zip(list(df_recipe_cost['Name']), quantities_lunch))
output_dinner = dict(zip(list(df_recipe_cost['Name']), quantities_dinner))

# Filter out dishes with a number to make of 0.0
output_breakfast = {dish: quantity for dish, quantity in output_breakfast.items() if quantity > 0}
output_lunch = {dish: quantity for dish, quantity in output_lunch.items() if quantity > 0}
output_dinner = {dish: quantity for dish, quantity in output_dinner.items() if quantity > 0}
# Filter out dishes with a number to make of 0.0
output_breakfast = {dish: quantity for dish, quantity in output_breakfast.items() if quantity > 0}
output_lunch = {dish: quantity for dish, quantity in output_lunch.items() if quantity > 0}
output_dinner = {dish: quantity for dish, quantity in output_dinner.items() if quantity > 0}

# Calculate time spent for each meal
time_spent_breakfast = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_breakfast[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_lunch = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_lunch[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_dinner = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * quantities_dinner[i] for i, recipe in enumerate(df_recipe_cost['Name']))

# Print out time spent for each meal
print("Time spent for breakfast:", round(time_spent_breakfast, 2), "minutes")
print("Time spent for lunch:", round(time_spent_lunch, 2), "minutes")
print("Time spent for dinner:", round(time_spent_dinner, 2), "minutes")


 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  167
   Intermediates:  0
   Connections  :  0
   Equations    :  30
   Residuals    :  30
 
 Number of state variables:    167
 Number of total equations: -  29
 Number of slack variables: -  29
 ---------------------------------------
 Degrees of freedom       :    109
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.07 NLPi:    6 Dpth:    0 Lvs:    3 Obj: -2.92E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.03 NLPi:    3 Dpth:    1 Lvs:    2 Obj: -2.92E+01 Gap:       NaN
Iter:     3 I:  0 Tm:      0.03 NLPi:    3 Dpth:    1 Lvs:    4 Obj: -2.92E+01 Ga

In [48]:
print("breakfast")
print(output_breakfast)
print("lunch")
print(output_lunch)
print("dinner")

print(output_dinner)

breakfast
{'Broccolini Quinoa Pilaf': 1.0}
lunch
{'Slow Cooker Chicken and Dumplings': 3.0}
dinner
{'Broccolini Quinoa Pilaf': 1.0, 'Swiss Chard Wraps': 1.0, 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}


In [46]:
print("breakfast")
print(output_breakfast)
print("lunch")
print(output_lunch)
print("dinner")

print(output_dinner)

breakfast
{'Broccolini Quinoa Pilaf': 1.0}
lunch
{'Slow Cooker Chicken and Dumplings': 3.0}
dinner
{'Broccolini Quinoa Pilaf': 1.0, 'Swiss Chard Wraps': 1.0, 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}
